## 2. kakaotalk message 전송

### 1) scrapy 파일 생성

In [76]:
import scrapy, requests
from scrapy.http import TextResponse

In [75]:
!scrapy startproject covid_19_total_backup

Error: scrapy.cfg already exists in /home/ubuntu/python3/notebook/covid_19/covid_19_total_backup


### 2) 데이터 크롤링 확인

#### 코로나 바이러스 감염증

In [3]:
headers = {'user-agent':'Mozilla/5.0'}
req = requests.get('http://ncov.mohw.go.kr/',headers=headers)
response = TextResponse(req.url, body=req.text, encoding='utf-8')
response

<200 http://ncov.mohw.go.kr/>

In [4]:
date = response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/h2/a/span[1]/text()').extract()
date = list(date)[0].split(" ") 
date = date[0:3]
date = " ".join(date)
date = date.replace("(","21.").replace(",","")
country_in =list(map(int, response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/div[1]/div/ul/li[1]/span[2]/text()').extract()))
country_out =list(map(int, response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/div[1]/div/ul/li[2]/span[2]/text()').extract()))
total_country = [country_in+country_out for country_in,country_out in zip(country_in,country_out)][0]
capital_distance = response.xpath('//*[@id="main_maplayout"]/button[9]/span[2]/text()').extract()[0]
noncapital_distance = response.xpath('//*[@id="main_maplayout"]/button[17]/span[2]/text()').extract()[0]

date,country_in, country_out, total_country,capital_distance,noncapital_distance


('21.3.15. 00시 기준', [370], [12], 382, '2', '1.5')

#### 코로나19 예방접종

In [3]:
headers = {'user-agent':'Mozilla/5.0'}
req = requests.get('http://ncv.kdca.go.kr/',headers=headers)
response = TextResponse(req.url, body=req.text, encoding='utf-8')
response

<200 http://ncv.kdca.go.kr/>

In [7]:
date = response.xpath('//*[@id="content"]/div[2]/div/h2/span[2]/em/text()').extract()
date = list(date)[0].split(" ") 
date = " ".join(date)
date = date.replace(",","")
date = '21.' + date

take_yesterday_1 = response.xpath('//*[@id="content"]/div[2]/div/div/div[1]/div[1]/ul/li[2]/div/p/span/text()').extract()
take_yesterday_1 = list(take_yesterday_1)[0]
take_yesterday_2 = response.xpath('//*[@id="content"]/div[2]/div/div/div[1]/div[2]/ul/li[2]/div/p/span/text()').extract()

total_1 = response.xpath('//*[@id="content"]/div[2]/div/div/div[1]/div[1]/ul/li[1]/div/p/span[1]/text()').extract()
total_2 = response.xpath('//*[@id="content"]/div[2]/div/div/div[1]/div[2]/ul/li[1]/div/p/span[1]/text()').extract()

seoul_vaccine = response.xpath('//*[@id="content"]/div[2]/div/div/div[2]/div/div[1]/div[2]/div[2]/div/p[1]/span[2]/text()').extract()
gyeonggi_vaccine = response.xpath('//*[@id="content"]/div[2]/div/div/div[2]/div/div[1]/div[2]/div[2]/div/p[9]/span[2]/text()').extract()
incheon_vaccine = response.xpath('//*[@id="content"]/div[2]/div/div/div[2]/div/div[1]/div[2]/div[2]/div/p[4]/span[2]/text()').extract()

date,take_yesterday_1,take_yesterday_2,total_1,total_2,seoul_vaccine, gyeonggi_vaccine, incheon_vaccine

('21.3.16. 0시 기준',
 '11,922',
 ['0'],
 ['602,150'],
 ['0'],
 ['3,216'],
 ['2,875'],
 ['501'])

### 3) items.py 파일 생성

In [40]:
%%writefile covid_19_total_backup/covid_19_total_backup/items.py
import scrapy


class Covid19TotalBackupItem(scrapy.Item):
    date = scrapy.Field()
    country_in = scrapy.Field()
    country_out = scrapy.Field()
    total_country = scrapy.Field()
    capital_distance = scrapy.Field()
    noncapital_distance = scrapy.Field()
    take_yesterday_1 = scrapy.Field()
    take_yesterday_2 = scrapy.Field()
    total_1 = scrapy.Field()
    total_2 = scrapy.Field()
    seoul_vaccine = scrapy.Field()
    gyeonggi_vaccine = scrapy.Field()
    incheon_vaccine = scrapy.Field()

Overwriting covid_19_total_backup/covid_19_total_backup/items.py


### 4) spider.py 생성

In [66]:
%%writefile  covid_19_total_backup/covid_19_total_backup/spiders/spider.py
import scrapy,re
from scrapy.crawler import CrawlerProcess
from covid_19_total_backup.items import Covid19TotalBackupItem

class Spider(scrapy.Spider):
    name = "covid_19_total_backup"
    start_urls =  ["http://ncov.mohw.go.kr/",
                   'http://ncv.kdca.go.kr/']
    
    def start_requests(self):  
        yield scrapy.Request("http://ncov.mohw.go.kr/", callback=self.parse_content)
        yield scrapy.Request('http://ncv.kdca.go.kr/', callback=self.parse)
        

    def parse_content(self,response):
        item = Covid19TotalBackupItem()
        date = response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/h2/a/span[1]/text()').extract()
        date = list(date)[0].split(" ") 
        date = date[0:3]
        date = " ".join(date)
        item['date'] = date.replace("(","21.").replace(",","")
        item['country_in'] =list(map(int, response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/div[1]/div/ul/li[1]/span[2]/text()').extract()))
        item['country_out'] =list(map(int, response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/div[1]/div/ul/li[2]/span[2]/text()').extract()))
        item['total_country'] = [item['country_in']+item['country_out'] for item['country_in'],item['country_out'] in zip(item['country_in'],item['country_out'])][0]
        item['capital_distance'] = response.xpath('//*[@id="main_maplayout"]/button[9]/span[2]/text()').extract()[0]
        item['noncapital_distance'] = response.xpath('//*[@id="main_maplayout"]/button[17]/span[2]/text()').extract()[0]
        yield item
    
    def parse(self, response):
        item = Covid19TotalBackupItem()
        item['take_yesterday_1'] = response.xpath('//*[@id="content"]/div[2]/div/div/div[1]/div[1]/ul/li[2]/div/p/span/text()').extract()
        item['take_yesterday_2']= response.xpath('//*[@id="content"]/div[2]/div/div/div[1]/div[2]/ul/li[2]/div/p/span/text()').extract()

        item['total_1'] = response.xpath('//*[@id="content"]/div[2]/div/div/div[1]/div[1]/ul/li[1]/div/p/span[1]/text()').extract()
        item['total_2'] = response.xpath('//*[@id="content"]/div[2]/div/div/div[1]/div[2]/ul/li[1]/div/p/span[1]/text()').extract()

        item['seoul_vaccine'] = response.xpath('//*[@id="content"]/div[2]/div/div/div[2]/div/div[1]/div[2]/div[2]/div/p[1]/span[2]/text()').extract()
        item['gyeonggi_vaccine'] = response.xpath('//*[@id="content"]/div[2]/div/div/div[2]/div/div[1]/div[2]/div[2]/div/p[9]/span[2]/text()').extract()
        item['incheon_vaccine'] = response.xpath('//*[@id="content"]/div[2]/div/div/div[2]/div/div[1]/div[2]/div[2]/div/p[4]/span[2]/text()').extract()

        yield item

Overwriting covid_19_total_backup/covid_19_total_backup/spiders/spider.py


### 5) shell script 파일 생성

In [36]:
%pwd

'/home/ubuntu/python3/notebook/covid_19'

In [68]:
%%writefile run.sh
cd /home/ubuntu/python3/notebook/covid_19/covid_19_total_backup/
scrapy crawl covid_19_total_backup

Overwriting run.sh


In [69]:
!/bin/bash run.sh

2021-03-15 05:33:05 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: covid_19_total_backup)
2021-03-15 05:33:05 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Dec 25 2020, 07:37:56) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Linux-5.4.0-1038-aws-x86_64-with-debian-buster-sid
2021-03-15 05:33:05 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-15 05:33:05 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'covid_19_total_backup',
 'NEWSPIDER_MODULE': 'covid_19_total_backup.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['covid_19_total_backup.spiders']}
2021-03-15 05:33:05 [scrapy.extensions.telnet] INFO: Telnet Password: e0e374c7bae59339
2021-03-15 05:33:05 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetCon

2021-03-15 05:33:11 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:56765/session/91b67acedb40f5f5c957c85cb0cf00e2/element {"using": "css selector", "value": "[id=\"id_email_2\"]"}
2021-03-15 05:33:11 [urllib3.connectionpool] DEBUG: http://127.0.0.1:56765 "POST /session/91b67acedb40f5f5c957c85cb0cf00e2/element HTTP/1.1" 200 88
2021-03-15 05:33:11 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-15 05:33:11 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:56765/session/91b67acedb40f5f5c957c85cb0cf00e2/element/3084a416-0650-4f4e-b043-3b47b8b0818a/clear {"id": "3084a416-0650-4f4e-b043-3b47b8b0818a"}
2021-03-15 05:33:11 [urllib3.connectionpool] DEBUG: http://127.0.0.1:56765 "POST /session/91b67acedb40f5f5c957c85cb0cf00e2/element/3084a416-0650-4f4e-b043-3b47b8b0818a/clear HTTP/1.1" 200 14
2021-03-15 05:33:11 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-15 05:33:11 [selenium.web

2021-03-15 05:33:26 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:45799/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "goog:chromeOptions": {"extensions": [], "args": ["headless", "window-size=1920x1080", "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36"]}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "goog:chromeOptions": {"extensions": [], "args": ["headless", "window-size=1920x1080", "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36"]}}}
2021-03-15 05:33:26 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1:45799
2021-03-15 05:33:27 [urllib3.connectionpool] DEBUG: http://127.0.0.1:45799 "POST /session HTTP/1.1" 200 680
2021-03-15 05:33:27 [selenium.webdriver.remote.remote_connection]

2021-03-15 05:33:44 [urllib3.connectionpool] DEBUG: http://127.0.0.1:45799 "DELETE /session/8010873b8d03494e864055f063820f94/window HTTP/1.1" 200 12
2021-03-15 05:33:44 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-15 05:33:44 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): kauth.kakao.com:443
2021-03-15 05:33:45 [urllib3.connectionpool] DEBUG: https://kauth.kakao.com:443 "POST /oauth/token HTTP/1.1" 200 None
2021-03-15 05:33:45 [scrapy.core.scraper] ERROR: Error processing {'gyeonggi_vaccine': ['744'],
 'incheon_vaccine': ['79'],
 'seoul_vaccine': ['1,704'],
 'take_yesterday_1': ['3,487'],
 'take_yesterday_2': ['0'],
 'total_1': ['587,884'],
 'total_2': ['0']}
Traceback (most recent call last):
  File "/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/twisted/internet/defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "/home/ubuntu/.pyenv/versions/3.6.9/env

### 6) kakaotalk message 전송을 위한 pipeline.py

In [84]:
%%writefile covid_19_total_backup/covid_19_total_backup/pipelines.py
from itemadapter import ItemAdapter
import requests, json
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import urllib.parse as urlparse
from urllib.parse import parse_qs
import random
from selenium.webdriver.chrome.options import Options
import time
import logging
from itemadapter import ItemAdapter


class Covid19TotalBackupPipeline:
    def process_item(self, item, spider):
        options = webdriver.ChromeOptions()
        options.add_argument("headless")
        options.add_argument("window-size=1920x1080")
        options.add_argument('Mozilla/5.0')
        driver = webdriver.Chrome(options=options)

        REST_API_KEY = "REST_API_KEY"
        REDIRECT_URL = "REDIRECT_URL/oauth"

        url = "https://kauth.kakao.com/oauth/authorize?client_id={}&redirect_uri={}&response_type=code".format(REST_API_KEY, REDIRECT_URL)
        driver.get(url)
        time.sleep(2)

        elem_login = driver.find_element_by_id("id_email_2")
        elem_login.clear()
        elem_login.send_keys("id_email1")
        time.sleep(0.3)
        elem_login.send_keys("id_email2")
        time.sleep(0.6)
        elem_login.send_keys("id_email3")
        time.sleep(0.5)
        elem_login.send_keys("id_email4")
        time.sleep(0.7)
        elem_login.send_keys("id_email5")
        time.sleep(2)

        elem_login = driver.find_element_by_id("id_password_3")
        elem_login.clear()
        elem_login.send_keys("password") 
        time.sleep(2.8)

        xpath = """//*[@id="login-form"]/fieldset/div[8]/button[1]"""
        driver.find_element_by_xpath(xpath).click()

        time.sleep(6)

        wait_time = 60

        driver.implicitly_wait(wait_time)
        url_current = driver.current_url

        parsed = urlparse.urlparse(url_current)
        access_token = parse_qs(parsed.query)

        driver.close()

        auth_code = access_token
        auth_code = auth_code['code'][0]
        url = "https://kauth.kakao.com/oauth/token" #post
        params ={
            "grant_type":"authorization_code",
            "client_id":REST_API_KEY,
            "redirect_uri":REDIRECT_URL,
            "code": auth_code,
        }

        response = requests.post(url, params)

        user_datas = response.json()

        ACCESS_TOKEN = user_datas["access_token"]

        url = "https://kapi.kakao.com/v2/user/me"

        headers = {"Authorization" : f'Bearer {ACCESS_TOKEN}'}

        url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"
        params = {
           "object_type":"text",
            "text" : '<코로나 알리미>\n\n{0}\n\n전체 확진자 : {1}\n국내 : {2}\n해외 유입 : {3}\n\n수도권 거리두기 : {4} 단계\n비수도권 거리두기 : {5} 단계\n\n백신현황 : {6}'.format(str(item['date']), str(item['total_country']),str(item['country_in']),str(item['country_out']),str(item['capital_distance']),str(item['noncapital_distance']),str(item['take_yesterday_1'])), 
            "link":{
                "web_url":"http://ncov.mohw.go.kr/",
                "mobile_web_url":"http://ncov.mohw.go.kr/",
            },
            "button_title" : "바로 클릭하세요!!",    
        }

        headers["Content-Type"] = "application/x-www-form-urlencoded"

        payload = "template_object=" + json.dumps(params)

        response = requests.post(url, payload, headers=headers) 
        
        return item

Overwriting covid_19_total_backup/covid_19_total_backup/pipelines.py


In [69]:
!/bin/bash run.sh

2021-03-15 05:33:05 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: covid_19_total_backup)
2021-03-15 05:33:05 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Dec 25 2020, 07:37:56) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Linux-5.4.0-1038-aws-x86_64-with-debian-buster-sid
2021-03-15 05:33:05 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-15 05:33:05 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'covid_19_total_backup',
 'NEWSPIDER_MODULE': 'covid_19_total_backup.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['covid_19_total_backup.spiders']}
2021-03-15 05:33:05 [scrapy.extensions.telnet] INFO: Telnet Password: e0e374c7bae59339
2021-03-15 05:33:05 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetCon

2021-03-15 05:33:11 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:56765/session/91b67acedb40f5f5c957c85cb0cf00e2/element {"using": "css selector", "value": "[id=\"id_email_2\"]"}
2021-03-15 05:33:11 [urllib3.connectionpool] DEBUG: http://127.0.0.1:56765 "POST /session/91b67acedb40f5f5c957c85cb0cf00e2/element HTTP/1.1" 200 88
2021-03-15 05:33:11 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-15 05:33:11 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:56765/session/91b67acedb40f5f5c957c85cb0cf00e2/element/3084a416-0650-4f4e-b043-3b47b8b0818a/clear {"id": "3084a416-0650-4f4e-b043-3b47b8b0818a"}
2021-03-15 05:33:11 [urllib3.connectionpool] DEBUG: http://127.0.0.1:56765 "POST /session/91b67acedb40f5f5c957c85cb0cf00e2/element/3084a416-0650-4f4e-b043-3b47b8b0818a/clear HTTP/1.1" 200 14
2021-03-15 05:33:11 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-15 05:33:11 [selenium.web

2021-03-15 05:33:26 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:45799/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "goog:chromeOptions": {"extensions": [], "args": ["headless", "window-size=1920x1080", "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36"]}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "goog:chromeOptions": {"extensions": [], "args": ["headless", "window-size=1920x1080", "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36"]}}}
2021-03-15 05:33:26 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1:45799
2021-03-15 05:33:27 [urllib3.connectionpool] DEBUG: http://127.0.0.1:45799 "POST /session HTTP/1.1" 200 680
2021-03-15 05:33:27 [selenium.webdriver.remote.remote_connection]

2021-03-15 05:33:44 [urllib3.connectionpool] DEBUG: http://127.0.0.1:45799 "DELETE /session/8010873b8d03494e864055f063820f94/window HTTP/1.1" 200 12
2021-03-15 05:33:44 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-03-15 05:33:44 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): kauth.kakao.com:443
2021-03-15 05:33:45 [urllib3.connectionpool] DEBUG: https://kauth.kakao.com:443 "POST /oauth/token HTTP/1.1" 200 None
2021-03-15 05:33:45 [scrapy.core.scraper] ERROR: Error processing {'gyeonggi_vaccine': ['744'],
 'incheon_vaccine': ['79'],
 'seoul_vaccine': ['1,704'],
 'take_yesterday_1': ['3,487'],
 'take_yesterday_2': ['0'],
 'total_1': ['587,884'],
 'total_2': ['0']}
Traceback (most recent call last):
  File "/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/twisted/internet/defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "/home/ubuntu/.pyenv/versions/3.6.9/env